In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import lightgbm as lg
%matplotlib inline

/home/divyam/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train=pd.read_csv('covid_19_train.csv')
test=pd.read_csv('covid_19_test.csv')
submission=pd.read_csv('covid_19_submission.csv')

train.head()
df=pd.concat([train,test])
train.info()
train.describe()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 111 entries, id to ctO2 (arterial blood gas analysis)
dtypes: float64(89), int64(5), object(17)
memory usage: 3.4+ MB


(5644, 111)

In [3]:
df.isna().mean(0).sort_values()
df=df.drop((df.isna().mean(0)).index[df.isna().mean(0)>0.98].values,axis='columns')
df

,id,age_quantile,covid_19,Patient addmited to regular ward,Patient addmited to semi-intensive unit,Patient addmited to intensive care unit,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Alkaline phosphatase,Strepto A,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),International normalized ratio (INR)
0,f60118eba52d74a,15,0.0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,024a42e8ac351fe,0,0.0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6eb7cb571bdedaf,5,0.0,0,0,0,2.090489,2.045308,-0.768649,1.469188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a038ae69a3f0de9,16,0.0,0,0,0,0.579844,0.416252,-0.630469,-0.999063,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6728175fd3bb714,6,0.0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,5ba48fd9d5dd05e,19,NaN,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1640,731a1b637d73fac,4,NaN,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1641,30fe0388aafc474,9,NaN,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642,5c1421eb95ab55c,18,NaN,0,0,0,-0.083925,-0.147652,-0.442042,0.459449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


df=df.dropna(thresh=df.shape[0]*0.025,how='all',axis=1)
df

In [4]:
df.dtypes

id                                          object
age_quantile                                 int64
covid_19                                   float64
Patient addmited to regular ward             int64
Patient addmited to semi-intensive unit      int64
                                            ...   
pO2 (venous blood gas analysis)            float64
Total CO2 (venous blood gas analysis)      float64
pH (venous blood gas analysis)             float64
HCO3 (venous blood gas analysis)           float64
International normalized ratio (INR)       float64
Length: 62, dtype: object

In [5]:
df=df.fillna((df.median(axis=0)))
df

,id,age_quantile,covid_19,Patient addmited to regular ward,Patient addmited to semi-intensive unit,Patient addmited to intensive care unit,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Alkaline phosphatase,Strepto A,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),International normalized ratio (INR)
0,f60118eba52d74a,15,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
1,024a42e8ac351fe,0,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
2,6eb7cb571bdedaf,5,0.0,0,0,0,2.090489,2.045308,-0.768649,1.469188,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
3,a038ae69a3f0de9,16,0.0,0,0,0,0.579844,0.416252,-0.630469,-0.999063,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
4,6728175fd3bb714,6,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,5ba48fd9d5dd05e,19,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
1640,731a1b637d73fac,4,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
1641,30fe0388aafc474,9,0.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
1642,5c1421eb95ab55c,18,0.0,0,0,0,-0.083925,-0.147652,-0.442042,0.459449,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144


In [6]:
df[df['covid_19']>0]

,id,age_quantile,covid_19,Patient addmited to regular ward,Patient addmited to semi-intensive unit,Patient addmited to intensive care unit,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Alkaline phosphatase,Strepto A,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),International normalized ratio (INR)
14,44c16f1e79380b4,8,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
32,fd44054b4168e33,7,1.0,0,0,0,0.076295,0.290940,-0.655593,0.459449,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
35,21be2fec728542e,9,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
37,1e7e33688edc5a9,11,1.0,1,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
49,f0ee30f04652697,11,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972,3918421c87c971e,19,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
3974,797d8250330ebdb,9,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
3978,72a12cea73b65ba,19,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144
3984,bc01e5089e9988f,7,1.0,0,0,0,0.053407,0.040316,-0.121716,-0.101517,...,-0.3576,0.0,0.013734,0.090369,0.080306,-0.21263,0.104088,-0.090881,0.100957,-0.156144


In [7]:
ytrain=df.iloc[:4000,2].values
ytest=df.iloc[4000:,2].values

In [8]:
dfit=df.drop('covid_19',axis='columns')
x=dfit.drop('id',axis='columns')
xtrain=x.iloc[:4000,:].values
xtest=x.iloc[4000:,:].values
xtrain



array([[15.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419],
       [ 0.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419],
       [ 5.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419],
       ...,
       [ 7.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419],
       [14.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419],
       [14.        ,  0.        ,  0.        , ..., -0.09088095,
         0.10095738, -0.15614419]])

from sklearn.linear_model import LogisticRegression
mod=LogisticRegression(solver='saga',C=30,random_state=0)
mod.fit(xtrain,ytrain)

mod.predict_proba(xtest)
mod.score(xtrain,ytrain)

from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(
    max_depth=10, 
    min_samples_leaf=28, 
    max_features=0.94, 
    criterion="gini",                                    
    random_state=1)

dt = tree_clf.fit(xtrain, ytrain)
dt_pred = dt.predict(xtest)
tree_clf.score(xtrain,ytrain)

from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(dt_pred,ytest))

print(classification_report(dt_pred,ytest))


from sklearn.ensemble import AdaBoostClassifier

rfm=AdaBoostClassifier()
d=rfm.fit(xtrain,ytrain)
score=rfm.score(xtrain,ytrain)
ada_pred=d.predict(xtest)


print(confusion_matrix(ada_pred,ytest))


print(classification_report(ada_pred,ytest))

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfm=RandomForestClassifier(n_estimators=50,random_state=40,min_samples_leaf=30,max_features=23)
d=rfm.fit(xtrain,ytrain)
score=rfm.score(xtrain,ytrain)
rf_pred=d.predict(xtest)
print(score)
print(rf_pred)
imp = pd.DataFrame({
    'colnames': x.columns,
    'importances': rfm.feature_importances_
})

good_columns = imp.loc[imp['importances'] != 0, :].colnames.values
imp

0.8985
[0. 0. 0. ... 0. 0. 0.]


,colnames,importances
0,age_quantile,0.298450
1,Patient addmited to regular ward,0.063697
2,Patient addmited to semi-intensive unit,0.000000
3,Patient addmited to intensive care unit,0.000000
4,Hematocrit,0.010835
5,Hemoglobin,0.016365
6,Platelets,0.172270
7,Mean platelet volume,0.003793
8,Red blood Cells,0.014946
9,Lymphocytes,0.005658


In [20]:
train_X = df.iloc[:4000, :].loc[:, good_columns].values
test_X = df.iloc[4000:, :].loc[:, good_columns].values

In [24]:
bt = lg.LGBMClassifier()

bt.fit(train_X,ytrain)
rf__pred=bt.predict_proba(test_X)[:,1]
bt.score(train_X,ytrain)
rfpred=bt.predict(test_X)

In [25]:
submission['covid_19']=bt.predict_proba(test_X)[:,1]
submission


,id,covid_19
0,4b8e26ec9ac6a73,0.165020
1,72ca7ed61c2a196,0.170518
2,400aecdd7f1a825,0.097511
3,b1ae71a4fe1788a,0.094924
4,654c1ae408ea3ac,0.170518
...,...,...
1639,5ba48fd9d5dd05e,0.152836
1640,731a1b637d73fac,0.138386
1641,30fe0388aafc474,0.170518
1642,5c1421eb95ab55c,0.002700


In [26]:
submission.to_csv('covid_19_submission.csv',index=False)


In [27]:
pd.read_csv('covid_19_submission.csv')

,id,covid_19
0,4b8e26ec9ac6a73,0.165020
1,72ca7ed61c2a196,0.170518
2,400aecdd7f1a825,0.097511
3,b1ae71a4fe1788a,0.094924
4,654c1ae408ea3ac,0.170518
...,...,...
1639,5ba48fd9d5dd05e,0.152836
1640,731a1b637d73fac,0.138386
1641,30fe0388aafc474,0.170518
1642,5c1421eb95ab55c,0.002700
